**Introduction:**

The script delineated below encompasses a series of operations aimed at geospatial analysis and visualization utilizing various Python libraries and Google Earth Engine (GEE). The primary focus is on extracting, processing, and visualizing geospatial data over a specified region and time frame. Notable functionalities include reading geometries from files, converting geometries to GEE format, filtering datasets based on geographical bounds and time, and visualizing and exporting monthly mean Earth Engine images using Folium maps.

**Import Statements:**
Importing necessary libraries and modules for data manipulation, geospatial analysis, and visualization:

In [ ]:
!pip install rasterio
import ee
import pandas as pd
import folium
import os
import rasterio
import geopandas as gpd
import shapely

**Earth Engine Initialization:**
Authentication and initialization of the Google Earth Engine API

In [ ]:
ee.Authenticate()
ee.Initialize()


Function: **read_geometry**

Reading a geometry from a file (e.g., GeoJSON, Shapefile) and returning a specified feature's geometry:

In [ ]:
def read_geometry(filepath, feature_idx=0):
    """
    Reads a file (GeoJSON, Shapefile, etc.) and returns a specific feature geometry.

    Parameters:
        filepath (str): Path to the file.
        feature_idx (int): Index of the feature to extract. Defaults to 0.

    Returns:
        geometry (shapely.geometry): Geometry of the specified feature.
    """
    try:
        geo_data = gpd.read_file(filepath)
        geometry = geo_data.geometry.iloc[feature_idx]
        return geometry
    except FileNotFoundError:
        raise ValueError(f"File not found: {filepath}")
    except IndexError:
        raise ValueError(f"Feature index {feature_idx} out of range")

Function: **shapely_to_ee**

Converting Shapely geometry to Earth Engine geometry

In [ ]:
def shapely_to_ee(geometry):
    """
    Converts a Shapely geometry to an Earth Engine geometry.

    Parameters:
        geometry (shapely.geometry): Geometry to convert.

    Returns:
        ee_geometry (ee.Geometry): Converted Earth Engine geometry.
        geo_coords (list): List of coordinates suitable for the "region" parameter in Earth Engine export methods.
    """
    if isinstance(geometry, shapely.geometry.multipolygon.MultiPolygon):
        ee_polygons = [ee.Geometry.Polygon(list(polygon.exterior.coords)) for polygon in geometry.geoms]
        ee_geometry = ee.Geometry.MultiPolygon(ee_polygons)
        # Extract coordinates from the first polygon as an example
        # Note: You might want to adjust this to fit your specific use case
        geo_coords = [list(coord) for coord in geometry.geoms[0].exterior.coords]
    elif isinstance(geometry, shapely.geometry.polygon.Polygon):
        ee_geometry = ee.Geometry.Polygon(list(geometry.exterior.coords))
        geo_coords = [list(coord) for coord in geometry.exterior.coords]
    else:
        raise ValueError(f"Unsupported geometry type: {type(geometry)}")

    return ee_geometry, geo_coords

**Geometry Reading and Conversion:**

Loading a specific geometry from a file and converting it to Earth Engine geometry. Data source for the shapefile: Delhi_Boundary.shp by Data Meet India community (https://datameet.org/) (CC BY 4.0)

In [ ]:
filepath = '/content/Delhi_Boundary.shp'
feature_idx = 0
geometry = read_geometry(filepath, feature_idx)
ee_geometry, geo_coords = shapely_to_ee(geometry)

**Dataset Filtering:**

Defining and filtering a dataset for a specific region and time period. Dataset provider: ***European Union/ESA/Copernicus***

In [ ]:
dataset = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_AER_AI')
filtered_data = dataset.filterBounds(ee_geometry).filterDate(ee.Date('2018-07-11'), ee.Date('2023-09-14'))


Function: **calculate_monthly_mean**

Calculating the monthly mean for a specified band, region, and time period

In [ ]:
def calculate_monthly_mean(filtered_data, ee_geometry, start_date, end_date, band_name=None):
    # If band_name is not provided, automatically detect it
    if not band_name:
        # Get the band names from the first image in the dataset
        first_image = filtered_data.first()
        available_bands = first_image.bandNames().getInfo()

        # If there's only one band, use it. Otherwise, raise an error.
        if len(available_bands) == 1:
            band_name = available_bands[0]
        else:
            raise ValueError(f"Multiple bands detected: {available_bands}. Please specify the band_name argument.")

    monthly_data = filtered_data.filterDate(ee.Date(start_date), ee.Date(end_date))
    monthly_mean = monthly_data.select(band_name).mean().clip(ee_geometry)
    return monthly_mean

Function: **calculate_monthly_means**

Looping through specified years and months to calculate monthly mean values for each month

In [ ]:
def calculate_monthly_means(filtered_data, ee_geometry, start_year=2018, end_year=2023, start_month=11, end_month=8, band_name=None):
    monthly_mean_images = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            # Adjust month range based on year
            month_start = start_month if year == start_year else 1
            month_end = end_month if year == end_year else 12

            if month < month_start or month > month_end:
                continue

            # Determine the number of days in the month
            days_in_month = 28
            if month == 2 and ((year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)):
                days_in_month = 29
            elif month in [1, 3, 5, 7, 8, 10, 12]:
                days_in_month = 31

            # Define start and end dates for each month
            start_date = f"{year}-{month:02}-01"
            end_date = f"{year}-{month:02}-{days_in_month}"

            # Calculate the monthly mean
            monthly_mean = calculate_monthly_mean(filtered_data, ee_geometry, start_date, end_date, band_name)
            monthly_mean_images.append(monthly_mean)

    return monthly_mean_images

Function: **visualize_monthly_mean_images_and_export**

Visualizing and exporting monthly mean images using Folium maps

In [ ]:
def visualize_monthly_mean_images_and_export(
    monthly_mean_images,
    output_directory,
    start_year=2018,
    start_month=11,
    end_year=2023,
    end_month=8,
    location_bounds=None,
    zoom_start=10,
    vis_params=None,
    label_location=None,
    name_prefix=None,
    display_maps=True
):
    """
    Visualizes a list of monthly mean Earth Engine images as Folium maps with customizable parameters and exports them as HTML files.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    current_year = start_year
    current_month = start_month

    for i, monthly_mean_image in enumerate(monthly_mean_images):
        # Create a Folium map centered on the specified location or the center of the image
        if location_bounds:
            m = folium.Map(location=[(location_bounds[0][0] + location_bounds[1][0]) / 2,
                                     (location_bounds[0][1] + location_bounds[1][1]) / 2],
                           zoom_start=zoom_start)
        else:
            # Get the center of the image's region and use it as the default location
            center = monthly_mean_image.geometry().centroid().getInfo()['coordinates']
            m = folium.Map(location=[center[1], center[0]], zoom_start=zoom_start)

        # Get the map ID for the monthly mean image
        vis_params = vis_params or {
            'min': 0,
            'max': 0.0005,
            'palette': ['blue', 'green', 'yellow', 'red']
        }
        map_id_dict = monthly_mean_image.getMapId(vis_params)

        # Add the image as a TileLayer to the map
        file_index = i + 1
        file_name = f"{file_index}_{current_month}_{current_year}"
        folium.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
            overlay=True,
            name=file_name,
        ).add_to(m)

        # Create a label with the month and year as numbers
        if label_location:
            folium.Marker(
                location=label_location,
                icon=folium.DivIcon(html=f"""<div style="background-color: transparent; padding: 5px; border-radius: 5px; font-size: 18px; font-weight: bold;">{current_month}/{current_year}</div>""")
            ).add_to(m)

        # Export the map as an HTML file
        filename = os.path.join(output_directory, f"{file_name}.html")
        m.save(filename)

        # Increment the month and year
        current_month += 1
        if current_month > 12:
            current_month = 1
            current_year += 1

        # Check if we've reached the end date
        if current_year > end_year or (current_year == end_year and current_month > end_month):
            break

        # Display the map interactively if specified
        if display_maps:
            display(m)

**Monthly Mean Calculation:**

Executing the monthly mean calculation for a specified time period and band

In [ ]:
start_year = 2018
end_year = 2023
start_month = 11
end_month = 8
# Using a different band name
monthly_mean_images = calculate_monthly_means(filtered_data, ee_geometry, start_year = start_year,
                                              end_year = end_year, start_month = start_month,
                                              end_month = end_month,
                                              band_name='absorbing_aerosol_index')

**Visualization and Export:**

Executing the visualization and export of monthly mean images with specified parameters

In [ ]:
output_directory = '/content/drive/MyDrive/Aerosol_sentinnel_near_real_time_images'

visualize_monthly_mean_images_and_export(
    monthly_mean_images,
    output_directory,
    start_year=2018,
    start_month=11,
    end_year=2023,
    end_month=8,
    location_bounds=[[28.4042, 76.8377], [28.8835, 77.2780]],  # Bounding box around New Delhi
    zoom_start=10,
    vis_params={
        'min': -1.5,
        'max': 1.8,
        'palette': ['blue', 'white', 'red']
    },
    label_location=[28.6139, 77.2090 + 0.20],  # New Delhi's coordinates for label placement
    name_prefix=None,
    display_maps=True
)

The script seamlessly melds functionalities from various libraries and Google Earth Engine to execute geospatial analysis and visualization tasks. Through a series of defined functions and procedures, it manages to read geometries, convert them to a compatible format, filter datasets based on specified criteria, calculate monthly mean values, and visualize the results interactively. This well-organized script is a valuable asset for anyone looking to explore geospatial data analysis and visualization in Python.